In [4]:
# from flask import Flask, request, render_template, make_response
# from flask_restful import Api, Resource
# from flask import send_file
# from flask import send_from_directory
# from pyowm import OWM
from weatherbit.api import Api as wApi
from datetime import datetime, timedelta
import pytz
import pandas as pd
from bokeh.palettes import plasma
from bokeh.plotting import figure, show
from bokeh.embed import components
from bokeh.models import DatetimeTickFormatter
import math
import os